In [17]:
import sqlite3
import numpy as np
import pandas as pd
from faker import Faker

# Number of samples
n = 1000

# Create SQLite database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Close any existing connections (if open)
conn.close()

# Reconnect to the database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Drop the existing tables
cursor.execute('DROP TABLE IF EXISTS Restaurants;')
cursor.execute('DROP TABLE IF EXISTS Dishes;')
cursor.execute('DROP TABLE IF EXISTS Customers;')
cursor.execute('DROP TABLE IF EXISTS Orders;')

# Create Restaurants table
cursor.execute('''
    CREATE TABLE Restaurants (
        Restaurant_ID INTEGER PRIMARY KEY,
        Restaurant_Name TEXT,
        Location TEXT
    );
''')

# Create Dishes table
cursor.execute('''
    CREATE TABLE Dishes (
        Dish_ID INTEGER PRIMARY KEY,
        Dish_Name TEXT,
        Dish_Category TEXT,
        Dish_Price REAL
    );
''')

# Create Customers table
cursor.execute('''
    CREATE TABLE Customers (
        Customer_ID INTEGER PRIMARY KEY,
        Customer_Name TEXT,
        Customer_Address TEXT
    );
''')

# Create Orders table
cursor.execute('''
    CREATE TABLE Orders (
        Order_ID INTEGER PRIMARY KEY,
        Customer_ID INTEGER,
        Restaurant_ID INTEGER,
        Dish_ID INTEGER,
        Quantity INTEGER,
        Order_Date TEXT,
        FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID),
        FOREIGN KEY (Restaurant_ID) REFERENCES Restaurants(Restaurant_ID),
        FOREIGN KEY (Dish_ID) REFERENCES Dishes(Dish_ID)
    );
''')

# Insert random data into Restaurants table
fake = Faker()
for i in range(n):
    cursor.execute('INSERT INTO Restaurants (Restaurant_Name, Location) VALUES (?, ?)',
                   (fake.company(), fake.city()))

# Insert random data into Dishes table
for i in range(n):
    cursor.execute('INSERT INTO Dishes (Dish_Name, Dish_Category, Dish_Price) VALUES (?, ?, ?)',
                   (fake.word(), fake.word(), np.random.uniform(5.0, 50.0)))

# Insert random data into Customers table
for i in range(n):
    cursor.execute('INSERT INTO Customers (Customer_Name, Customer_Address) VALUES (?, ?)',
                   (fake.name(), fake.address()))

# Insert random data into Orders table
for i in range(n):
    cursor.execute('INSERT INTO Orders (Customer_ID, Restaurant_ID, Dish_ID, Quantity, Order_Date) VALUES (?, ?, ?, ?, ?)',
                   (np.random.randint(1, n + 1), np.random.randint(1, n + 1), np.random.randint(1, n + 1),
                    np.random.randint(1, 5), fake.date_this_decade()))

# Commit changes and close connection
conn.commit()
conn.close()

# Example Query to check the number of rows and columns in the 'Orders' table
conn = sqlite3.connect('RestaurantMenuDB.db')
result = pd.read_sql_query('SELECT * FROM Orders;', conn)
conn.close()

# Display the shape of the result (number of rows and columns)
print("Number of Rows and Columns in the 'Orders' Table:")
print(result.shape)
print(result)


Number of Rows and Columns in the 'Orders' Table:
(1000, 6)
     Order_ID  Customer_ID  Restaurant_ID  Dish_ID  Quantity  Order_Date
0           1           63            638      501         3  2021-08-21
1           2          973            313      776         3  2022-02-16
2           3          634            830      502         2  2022-04-19
3           4          872            204       55         2  2021-07-04
4           5          199            429      951         3  2021-07-08
..        ...          ...            ...      ...       ...         ...
995       996          188             23      640         1  2023-08-03
996       997          268            815      499         3  2020-04-08
997       998          368            600      264         3  2021-11-27
998       999          329             22      142         2  2020-05-03
999      1000          420            226       55         2  2021-04-29

[1000 rows x 6 columns]


In [8]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Example Query 1: Retrieve Order Details with Customer and Dish Information
query1 = '''
    SELECT Orders.Order_ID, Customers.Customer_Name, Dishes.Dish_Name, Dishes.Dish_Category, Orders.Quantity, Orders.Order_Date
    FROM Orders
    INNER JOIN Customers ON Orders.Customer_ID = Customers.Customer_ID
    INNER JOIN Dishes ON Orders.Dish_ID = Dishes.Dish_ID;
'''

result1 = pd.read_sql_query(query1, conn)
print("Example Query 1: Retrieve Order Details with Customer and Dish Information")
print(result1)
print("\n")

# Example Query 2: Retrieve Restaurant Names with Total Orders
query2 = '''
    SELECT Restaurants.Restaurant_Name, COUNT(Orders.Order_ID) AS Total_Orders
    FROM Restaurants
    LEFT JOIN Orders ON Restaurants.Restaurant_ID = Orders.Restaurant_ID
    GROUP BY Restaurants.Restaurant_Name;
'''

result2 = pd.read_sql_query(query2, conn)
print("Example Query 2: Retrieve Restaurant Names with Total Orders")
print(result2)
print("\n")

# Example Query 3: Retrieve High-Value Orders with Dish Prices
query3 = '''
    SELECT Orders.Order_ID, Dishes.Dish_Name, Dishes.Dish_Price, Orders.Quantity, (Dishes.Dish_Price * Orders.Quantity) AS Total_Price
    FROM Orders
    INNER JOIN Dishes ON Orders.Dish_ID = Dishes.Dish_ID
    WHERE (Dishes.Dish_Price * Orders.Quantity) > 30;
'''

result3 = pd.read_sql_query(query3, conn)
print("Example Query 3: Retrieve High-Value Orders with Dish Prices")
print(result3)

# Close the connection
conn.close()


Example Query 1: Retrieve Order Details with Customer and Dish Information
     Order_ID     Customer_Name   Dish_Name Dish_Category  Quantity  \
0           1      Brandon Hall        draw            us         4   
1           2      Laurie Reyes     officer        choose         4   
2           3    Kenneth Jordan     program      actually         3   
3           4   Michelle Burton    question        others         3   
4           5  Nichole Johnston    Congress       feeling         2   
..        ...               ...         ...           ...       ...   
995       996     Kathy Collins       allow    Republican         3   
996       997    Timothy Jensen        want          talk         1   
997       998     Nathan Howard      arrive        choose         4   
998       999    Nicholas Lewis  television          town         4   
999      1000    Dylan Santiago        yard      daughter         2   

     Order_Date  
0    2022-12-07  
1    2021-07-22  
2    2020-07-06  


In [9]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Example Query: Joining Orders, Customers, Dishes, and Restaurants tables
query = '''
    SELECT Orders.Order_ID, Customers.Customer_Name, Dishes.Dish_Name, Dishes.Dish_Category,
           Restaurants.Restaurant_Name, Orders.Quantity, Orders.Order_Date
    FROM Orders
    INNER JOIN Customers ON Orders.Customer_ID = Customers.Customer_ID
    INNER JOIN Dishes ON Orders.Dish_ID = Dishes.Dish_ID
    INNER JOIN Restaurants ON Orders.Restaurant_ID = Restaurants.Restaurant_ID;
'''

result = pd.read_sql_query(query, conn)
print("Example Query: Joining Orders, Customers, Dishes, and Restaurants tables")
print(result)

# Close the connection
conn.close()


Example Query: Joining Orders, Customers, Dishes, and Restaurants tables
     Order_ID     Customer_Name   Dish_Name Dish_Category  \
0           1      Brandon Hall        draw            us   
1           2      Laurie Reyes     officer        choose   
2           3    Kenneth Jordan     program      actually   
3           4   Michelle Burton    question        others   
4           5  Nichole Johnston    Congress       feeling   
..        ...               ...         ...           ...   
995       996     Kathy Collins       allow    Republican   
996       997    Timothy Jensen        want          talk   
997       998     Nathan Howard      arrive        choose   
998       999    Nicholas Lewis  television          town   
999      1000    Dylan Santiago        yard      daughter   

                 Restaurant_Name  Quantity  Order_Date  
0                    Cole-Dawson         4  2022-12-07  
1      Johnson, Foster and Smith         4  2021-07-22  
2                    Mo

In [10]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Create a new table named CompleteOrders
create_table_query = '''
    CREATE TABLE IF NOT EXISTS CompleteOrders AS
    SELECT Orders.Order_ID, Customers.Customer_Name, Restaurants.Restaurant_Name,
           Dishes.Dish_Name, Dishes.Dish_Category, Orders.Quantity, Orders.Order_Date
    FROM Orders
    INNER JOIN Customers ON Orders.Customer_ID = Customers.Customer_ID
    INNER JOIN Dishes ON Orders.Dish_ID = Dishes.Dish_ID
    INNER JOIN Restaurants ON Orders.Restaurant_ID = Restaurants.Restaurant_ID;
'''

cursor.execute(create_table_query)

# Commit changes and close connection
conn.commit()
conn.close()


In [12]:
import sqlite3
import pandas as pd
import numpy as np
from faker import Faker

# Number of samples
n = 1000

# Generate random data
fake = Faker()
customer_names = [fake.name() for _ in range(n)]
restaurant_names = [fake.company() for _ in range(n)]
dish_names = [fake.word() for _ in range(n)]
dish_categories = ['Appetizer', 'Main Course', 'Dessert']
order_quantities = np.random.randint(1, 5, n)
order_dates = [fake.date_this_decade() for _ in range(n)]
# Drop the existing tables
cursor.execute('DROP TABLE IF EXISTS Restaurants;')
cursor.execute('DROP TABLE IF EXISTS Dishes;')
cursor.execute('DROP TABLE IF EXISTS Customers;')
cursor.execute('DROP TABLE IF EXISTS Orders;')
cursor.execute('DROP TABLE IF EXISTS MenuItems;')

# Create DataFrames
df_customers = pd.DataFrame({
    'Customer_ID': range(1, n + 1),
    'Customer_Name': customer_names,
    'Customer_Address': [fake.address() for _ in range(n)]
})

df_restaurants = pd.DataFrame({
    'Restaurant_ID': range(1, n + 1),
    'Restaurant_Name': restaurant_names,
    'Location': [fake.city() for _ in range(n)]
})

df_dishes = pd.DataFrame({
    'Dish_ID': range(1, n + 1),
    'Dish_Name': dish_names,
    'Dish_Category': np.random.choice(dish_categories, n)
})

df_orders = pd.DataFrame({
    'Order_ID': range(1, n + 1),
    'Customer_ID': np.random.randint(1, n + 1, n),
    'Restaurant_ID': np.random.randint(1, n + 1, n),
    'Dish_ID': np.random.randint(1, n + 1, n),
    'Quantity': order_quantities,
    'Order_Date': order_dates
})

# Create SQLite database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''
    CREATE TABLE Customers (
        Customer_ID INTEGER PRIMARY KEY,
        Customer_Name TEXT,
        Customer_Address TEXT
    );
''')

# Create Restaurants table
cursor.execute('''
    CREATE TABLE Restaurants (
        Restaurant_ID INTEGER PRIMARY KEY,
        Restaurant_Name TEXT,
        Location TEXT
    );
''')

# Create Dishes table
cursor.execute('''
    CREATE TABLE Dishes (
        Dish_ID INTEGER PRIMARY KEY,
        Dish_Name TEXT,
        Dish_Category TEXT
    );
''')

# Create Orders table
cursor.execute('''
    CREATE TABLE Orders (
        Order_ID INTEGER PRIMARY KEY,
        Customer_ID INTEGER,
        Restaurant_ID INTEGER,
        Dish_ID INTEGER,
        Quantity INTEGER,
        Order_Date TEXT,
        FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID),
        FOREIGN KEY (Restaurant_ID) REFERENCES Restaurants(Restaurant_ID),
        FOREIGN KEY (Dish_ID) REFERENCES Dishes(Dish_ID)
    );
''')

# Insert data into tables
df_customers.to_sql('Customers', conn, index=False, if_exists='replace')
df_restaurants.to_sql('Restaurants', conn, index=False, if_exists='replace')
df_dishes.to_sql('Dishes', conn, index=False, if_exists='replace')
df_orders.to_sql('Orders', conn, index=False, if_exists='replace')

# Commit changes and close connection
conn.commit()
conn.close()


In [13]:
import sqlite3
import pandas as pd

# Create SQLite database connection
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Create a new table 'CombinedData' by joining existing tables
query = '''
    CREATE TABLE CombinedData AS
    SELECT 
        Orders.Order_ID,
        Customers.Customer_Name,
        Customers.Customer_Address,
        Restaurants.Restaurant_Name,
        Restaurants.Location,
        Dishes.Dish_Name,
        Dishes.Dish_Category,
        Orders.Quantity,
        Orders.Order_Date
    FROM Orders
    INNER JOIN Customers ON Orders.Customer_ID = Customers.Customer_ID
    INNER JOIN Restaurants ON Orders.Restaurant_ID = Restaurants.Restaurant_ID
    INNER JOIN Dishes ON Orders.Dish_ID = Dishes.Dish_ID;
'''

cursor.execute(query)

# Commit changes and close connection
conn.commit()
conn.close()


In [14]:
import sqlite3
import numpy as np
import pandas as pd
from faker import Faker

# Number of samples
n = 1000

# Nominal data: Dish Categories
dish_categories = ['Appetizer', 'Main Course', 'Dessert']
dish_category_data = np.random.choice(dish_categories, n)

# Ordinal data: Dish Ratings
dish_ratings = ['Low', 'Medium', 'High']
dish_rating_data = np.random.choice(dish_ratings, n, p=[0.3, 0.4, 0.3])

# Interval data: Order Delivery Time
order_delivery_time = np.random.randint(10, 60, n)

# Ratio data: Dish Prices
dish_prices = np.random.uniform(5.0, 50.0, n)

# Create DataFrame for Dishes
df_dishes = pd.DataFrame({
    'Dish_Name': [fake.word() for _ in range(n)],
    'Dish_Category': dish_category_data,
    'Dish_Rating': dish_rating_data,
    'Dish_Price': dish_prices
})

# Create SQLite database
conn = sqlite3.connect('RestaurantMenuDB.db')
cursor = conn.cursor()

# Drop the existing Dishes, Orders, and Customers tables
cursor.execute('DROP TABLE IF EXISTS Dishes;')
cursor.execute('DROP TABLE IF EXISTS Orders;')
cursor.execute('DROP TABLE IF EXISTS Customers;')

# Create Dishes table
cursor.execute('''
    CREATE TABLE Dishes (
        Dish_ID INTEGER PRIMARY KEY,
        Dish_Name TEXT,
        Dish_Category TEXT,
        Dish_Rating TEXT,
        Dish_Price REAL
    );
''')

# Insert random data into Dishes table
for i in range(n):
    cursor.execute('INSERT INTO Dishes (Dish_Name, Dish_Category, Dish_Rating, Dish_Price) VALUES (?, ?, ?, ?)',
                   (df_dishes['Dish_Name'].iloc[i], df_dishes['Dish_Category'].iloc[i],
                    df_dishes['Dish_Rating'].iloc[i], df_dishes['Dish_Price'].iloc[i]))

# Create Customers table
cursor.execute('''
    CREATE TABLE Customers (
        Customer_ID INTEGER PRIMARY KEY,
        Customer_Name TEXT,
        Customer_Address TEXT
    );
''')

# Insert random data into Customers table
for i in range(n):
    cursor.execute('INSERT INTO Customers (Customer_Name, Customer_Address) VALUES (?, ?)',
                   (fake.name(), fake.address()))

# Create Orders table
cursor.execute('''
    CREATE TABLE Orders (
        Order_ID INTEGER PRIMARY KEY,
        Customer_ID INTEGER,
        Order_Date TEXT,
        Dish_ID INTEGER,
        Quantity INTEGER,
        Order_Delivery_Time INTEGER,
        Order_Status TEXT,
        FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID),
        FOREIGN KEY (Dish_ID) REFERENCES Dishes(Dish_ID)
    );
''')

# Insert random data into Orders table
for i in range(n):
    cursor.execute('INSERT INTO Orders (Customer_ID, Order_Date, Dish_ID, Quantity, Order_Delivery_Time, Order_Status) VALUES (?, ?, ?, ?, ?, ?)',
                   (np.random.randint(1, n+1), '2023-11-13', i+1, np.random.randint(1, 5),
                    df_dishes['Dish_Price'].iloc[i], 'Pending'))

# Commit changes and close connection
conn.commit()
conn.close()

# Example Query to check the number of rows and columns
conn = sqlite3.connect('RestaurantMenuDB.db')
result = pd.read_sql_query('SELECT * FROM Orders;', conn)
conn.close()

# Display the shape of the result (number of rows and columns)
print("Number of Rows and Columns in the 'Orders' Table:")
print(result.shape)
print(result)


Number of Rows and Columns in the 'Orders' Table:
(1000, 7)
     Order_ID  Customer_ID  Order_Date  Dish_ID  Quantity  \
0           1          616  2023-11-13        1         1   
1           2          282  2023-11-13        2         3   
2           3          634  2023-11-13        3         1   
3           4          951  2023-11-13        4         3   
4           5          211  2023-11-13        5         2   
..        ...          ...         ...      ...       ...   
995       996          400  2023-11-13      996         1   
996       997          622  2023-11-13      997         4   
997       998          252  2023-11-13      998         3   
998       999           67  2023-11-13      999         2   
999      1000          932  2023-11-13     1000         1   

     Order_Delivery_Time Order_Status  
0              15.213533      Pending  
1               6.666010      Pending  
2              17.179194      Pending  
3              12.378642      Pending  
4      

In [15]:
import sqlite3
import pandas as pd
import numpy as np
from faker import Faker

# Set up Faker for generating fake data
fake = Faker()

# Number of samples
n = 1000

# Generate random data for Customers
customer_data = {
    'Customer_ID': np.arange(1, n + 1),
    'Customer_Name': [fake.name() for _ in range(n)],
    'Customer_Email': [fake.email() for _ in range(n)],
    'Customer_Age': np.random.randint(18, 70, n),
    'Customer_Join_Date': [fake.date_this_decade() for _ in range(n)]
}

df_customers = pd.DataFrame(customer_data)

# Generate random data for Products
product_data = {
    'Product_ID': np.arange(1, n + 1),
    'Product_Name': [fake.word() for _ in range(n)],
    'Product_Category': [fake.word() for _ in range(n)],
    'Product_Price': np.random.uniform(5.0, 500.0, n),
    'Stock_Quantity': np.random.randint(1, 100, n)
}

df_products = pd.DataFrame(product_data)

# Generate random data for Orders
order_data = {
    'Order_ID': np.arange(1, n + 1),
    'Customer_ID': np.random.randint(1, n + 1, n),
    'Product_ID': np.random.randint(1, n + 1, n),
    'Order_Date': [fake.date_between(start_date='-1y', end_date='today') for _ in range(n)],
    'Order_Quantity': np.random.randint(1, 10, n),
    'Order_Total': np.random.uniform(10.0, 500.0, n),
    'Order_Shipped': [fake.boolean() for _ in range(n)]
}

df_orders = pd.DataFrame(order_data)

# Create SQLite database
conn = sqlite3.connect('SampleDatabase.db')
cursor = conn.cursor()

# Create Customers table
df_customers.to_sql('Customers', conn, index=False, if_exists='replace')

# Create Products table
df_products.to_sql('Products', conn, index=False, if_exists='replace')

# Create Orders table
df_orders.to_sql('Orders', conn, index=False, if_exists='replace')

# Commit changes and close connection
conn.commit()
conn.close()

# Example Query to check the number of rows and columns in the 'Orders' table
conn = sqlite3.connect('SampleDatabase.db')
result = pd.read_sql_query('SELECT * FROM Orders;', conn)
conn.close()

# Display the shape of the result (number of rows and columns)
print("Number of Rows and Columns in the 'Orders' Table:")
print(result.shape)
print(result)


Number of Rows and Columns in the 'Orders' Table:
(1000, 7)
     Order_ID  Customer_ID  Product_ID  Order_Date  Order_Quantity  \
0           1          550         771  2023-10-28               8   
1           2          983         336  2023-11-05               9   
2           3          661         758  2023-05-09               7   
3           4          461          60  2023-09-20               1   
4           5          440         315  2023-01-13               9   
..        ...          ...         ...         ...             ...   
995       996          415         252  2023-05-12               4   
996       997          793          39  2023-05-31               8   
997       998          864         542  2023-08-30               1   
998       999          981         293  2023-10-14               2   
999      1000          285         719  2023-09-24               2   

     Order_Total  Order_Shipped  
0     282.286312              1  
1     390.981004              0